First, get all packages that we might need and setup your notebook plotting to be inline.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pencil as pc

from matplotlib import animation, rc
from IPython.display import HTML

Get simulation objects and its extends, here I assume $N_x = N_y = N_z$

In [ ]:
sim = pc.get_sim()

L = sim.param['lxyz'][0]
Nx = sim.dim.nx

Then, read in varlist to get all var files in a list, plus I get the number range of all files and estimate there simulation time via dsnap.

In [ ]:
varlist = sim.get_varlist()
Nvar = np.size(varlist)
var_range = np.array([int(var[3:]) for var in varlist])
var_range_simtime = np.array([sim.get_value('dsnap')*varNum for varNum in var_range])

Now, lets read all the data we want to plot in >>one<< array:

In [7]:
print(Nvar)
rhop = np.zeros((Nvar, Nx, Nx))
for ii,var in zip(var_range, varlist):
    VAR = pc.read.var(sim=sim, trimall=True, quiet=True, var_file=var)
    rhop[ii] = VAR.rhop[0]

40


Create a figure and use imshow to plot your data. Make sure to use animated=True to get animation capability and use extent to set physically meaningfull plot ranges.

The function updatefig then updates the plot with a new snapshot by iterating the index ii.

Use intgervall to specify the ms for each frame and frames to specify the number of frames to be plotted.

In [ ]:
fig, ax = plt.subplots()

ii = -1
im = ax.imshow(rhop[100], animated=True, vmin=1e-2, vmax=10, extent=[-L, L, -L, L])

def updatefig(*args):
    global ii
    ii += 1
    if ii >= Nvar: return im,
    im.set_array(rhop[ii])
    return im,

ani = animation.FuncAnimation(fig, updatefig, interval=500, frames=Nvar-1)

Last step: lets have a look on the video.

In [ ]:
HTML(ani.to_html5_video())

And maybe we want to save it.

In [ ]:
mywriter = animation.FFMpegWriter(fps=10)
ani.save('tmp.mp4', writer=mywriter)